In [1]:
using Polymake
using Test

polymake version 4.9
Copyright (c) 1997-2023
Ewgenij Gawrilow, Michael Joswig, and the polymake team
Technische Universität Berlin, Germany
https://polymake.org

This is free software licensed under GPL; see the source for copying conditions.
There is NO warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [12]:
#Checking Polymake works properly
p = polytope.Polytope(POINTS=[1 -1 -1; 1 1 -1; 1 -1 1; 1 1 1; 1 0 0])
@show p.VERTICES

p.VERTICES = pm::Matrix<pm::Rational>
1 -1 -1
1 1 -1
1 -1 1
1 1 1



pm::Matrix<pm::Rational>
1 -1 -1
1 1 -1
1 -1 1
1 1 1


In [3]:
MAX_LATTICE_POINTS = 33 #global variable for the maximum number of lattice points
polytopes_list = [] #global list of polytopes

for i in 4:2:24
    open("SmoothGeneration_output/SmoothGeneration_output$i") do f #open the file
        number_polytopes = 0
        polytopes_read = 0
        while ! eof(f)
            #Read the number of vertices
            number_vertices = parse(Int, readline(f))
            polytopes_read += 1
            
            vertices_matrix = zeros(Int, number_vertices, 4)
            #Read the matrix of vertices of polytopes P
            for a in 1:number_vertices
                vertices_string = readline(f)
                vertex_vector = [1;parse.(Int64, split(vertices_string))] #Append 1 to the beginning due to polymake standards
                vertices_matrix[a, :] .= vertex_vector
            end
            #Create the polytope from its vertices
            new_polytope = polytope.Polytope(POINTS=vertices_matrix)
            
            @assert new_polytope.SMOOTH #Checks if the polytope is smooth
            if new_polytope.N_LATTICE_POINTS <= MAX_LATTICE_POINTS #Check if the polytope does not exceed the maximum number of lattice points
                isomorphic_found = false
                #Check we polytopes_list already has this polytope
                for smooth_polytope in polytopes_list
                    if polytope.lattice_isomorphic_smooth_polytopes(smooth_polytope, new_polytope) #Check if any of the polytopes are lattice-equivalent
                        isomorphic_found = true #Raise flag
                        break
                    end
                end
                if !isomorphic_found
                    push!(polytopes_list, new_polytope) #Append to the list
                    number_polytopes += 1
                end
            end
        end 
        println(number_polytopes, " smooth polytopes extracted of ", polytopes_read, " polytopes read from file")
    end
end

3 smooth polytopes extracted of 3 polytopes read from file
810 smooth polytopes extracted of 4168 polytopes read from file
519 smooth polytopes extracted of 10771 polytopes read from file
175 smooth polytopes extracted of 5591 polytopes read from file
206 smooth polytopes extracted of 17501 polytopes read from file
43 smooth polytopes extracted of 15673 polytopes read from file
19 smooth polytopes extracted of 9649 polytopes read from file
2 smooth polytopes extracted of 2428 polytopes read from file
0 smooth polytopes extracted of 563 polytopes read from file
0 smooth polytopes extracted of 32 polytopes read from file
0 smooth polytopes extracted of 17 polytopes read from file


In [4]:
#Outputting the Smooth 3-polytopes after creating the list
output_file = "prune_output" #filename

if !isfile(output_file)
    touch(output_file)  # Create an empty file if it doesn't exist
end

for smooth_polytope in polytopes_list
    @assert smooth_polytope.SMOOTH
    @assert smooth_polytope.NORMAL
    
    open(output_file, "a") do file #Open the file in appending mode
        println(file, smooth_polytope.N_VERTICES)

        #Printing the contents of the matrix and avoiding the Polymake types showing up
        for row in eachrow(smooth_polytope.VERTICES) 
            println(file, join(row[2:end], " "))
        end
    end
end

In [9]:
#Performing various analyses on the produced polytopes
counter = 0
counter1 = 0
counter2 = 0
counter3 = 0

for smooth_polytope in polytopes_list
    if smooth_polytope.N_INTERIOR_LATTICE_POINTS >= 1
        counter += 1
        if smooth_polytope.N_INTERIOR_LATTICE_POINTS == 1
            counter1 += 1
        end
        if smooth_polytope.N_INTERIOR_LATTICE_POINTS == 2
            counter2 += 1
        end
        if smooth_polytope.N_INTERIOR_LATTICE_POINTS == 3
            counter3 += 1
        end
    end
end

println(counter, " with an interior point")
println(counter1, " with one interior point")
println(counter2, " with two interior points")
println(counter3, " with three interior points")

println(size(polytopes_list))

146 with an interior point
58 with one interior point
85 with two interior points
3 with three interior points
(1777,)


In [6]:
p = polytopes_list[1600]
@show p.VERTICES
println(p.INTERIOR_LATTICE_POINTS)
p.VISUAL

p.VERTICES = pm::Matrix<pm::Rational>
1 0 0 0
1 0 1 0
1 2 0 0
1 3 1 0
1 3 2 0
1 1 2 0
1 0 0 1
1 0 1 1
1 4 0 1
1 6 2 1
1 6 3 1
1 2 3 1

pm::Matrix<pm::Integer>



LoadError: Exception occured at Polymake side:
unknown property Polytope<Rational>::VISUAL at /home/kyle/.julia/artifacts/51aeebd7aa37184cb3796181547a7148af3bd674/share/polymake/perllib/Polymake/Core/BigObjectType.pm line 432.
	Polymake::Core::BigObjectType::property(Polymake::Core::BigObjectType=ARRAY(0x70177d0), "VISUAL") called at /home/kyle/.julia/artifacts/51aeebd7aa37184cb3796181547a7148af3bd674/share/polymake/perllib/Polymake/Core/BigObjectType.pm line 714
	Polymake::Core::BigObjectType::encode_descending_path(Polymake::Core::BigObjectType=ARRAY(0x70177d0), "VISUAL") called at /home/kyle/.julia/artifacts/51aeebd7aa37184cb3796181547a7148af3bd674/share/polymake/perllib/Polymake/Core/BigObjectType.pm line 757
	Polymake::Core::BigObjectType::encode_read_request(Polymake::Core::BigObjectType=ARRAY(0x70177d0), "VISUAL") called at /home/kyle/.julia/artifacts/51aeebd7aa37184cb3796181547a7148af3bd674/share/polymake/perllib/Polymake/Core/BigObject.pm line 1551
	Polymake::Core::BigObject::